In [1]:
name = 'Xiaohan Wang'

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpl_colors
import ipywidgets

import bqplot

In [3]:
buildings = pd.read_csv('/Users/W/Downloads/building_inventory.csv',
                       na_values={'Square Footage':0,'Year Acquired':0,'Year Constructed':0})
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,In Use,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,In Use,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided


## Left component - Grid Heatmap


* Rows are congressional district
* Columns are the governmental department (Agency Name)
* Values are sum of total square footage for that set of criteria

In [4]:
# map each agency name into an unique ID(numerical)
agencies_unique = buildings['Agency Name'].unique()
n_agencies = len(agencies_unique)

agencies_dict = {}
for i in range(n_agencies):
    agencies_dict[agencies_unique[i]] = i
#agencies_dict

In [5]:
buildings['AgencyID'] = [agencies_dict[item] for item in buildings['Agency Name']] 

In [6]:
# get square footage of area in each congress district and each agency
temp_df = pd.DataFrame(buildings.groupby(['Congress Dist','Agency Name','AgencyID'])['Square Footage'].agg(['sum']))

In [7]:
temp_df.reset_index(inplace=True)
temp_df

,Congress Dist,Agency Name,AgencyID,sum
0,0,Department of Central Management Services,8,231278.0
1,0,Department of Human Services,2,372784.0
2,0,Department of Natural Resources,0,40249.0
3,0,Department of State Police,4,8239.0
4,0,Department of Transportation,3,55578.0
...,...,...,...,...
151,18,Department of Transportation,3,516516.0
152,18,Department of Veterans' Affairs,15,652160.0
153,18,Historic Preservation Agency,10,175427.0
154,18,Illinois Community College Board,23,252623.0


In [8]:
temp_df2 = pd.DataFrame(buildings.groupby(['Year Acquired','Congress Dist','AgencyID'])['Square Footage'].agg(['sum']))
temp_df2.reset_index(inplace=True)
temp_df2

,Year Acquired,Congress Dist,AgencyID,sum
0,1753.0,12,10,1200.0
1,1802.0,12,10,4440.0
2,1810.0,12,10,4033.0
3,1832.0,12,12,120000.0
4,1837.0,15,10,10302.0
...,...,...,...,...
2482,2017.0,4,3,6720.0
2483,2018.0,7,3,12000.0
2484,2018.0,17,3,2910.0
2485,2018.0,18,3,2250.0


In [9]:
df = temp_df2.merge(temp_df, on=['Congress Dist','AgencyID'])
df = df.rename(columns={"sum_x": "Yearly Total Square Footage", "sum_y": "Total Square Footage"})
df['Year Acquired'] = pd.to_datetime(df['Year Acquired'], format="%Y")
df

,Year Acquired,Congress Dist,AgencyID,Yearly Total Square Footage,Agency Name,Total Square Footage
0,1753-01-01,12,10,1200.0,Historic Preservation Agency,123924.0
1,1802-01-01,12,10,4440.0,Historic Preservation Agency,123924.0
2,1810-01-01,12,10,4033.0,Historic Preservation Agency,123924.0
3,1920-01-01,12,10,1425.0,Historic Preservation Agency,123924.0
4,1924-01-01,12,10,3200.0,Historic Preservation Agency,123924.0
...,...,...,...,...,...,...
2482,2002-01-01,7,13,15000.0,Illinois Medical District Commission,15000.0
2483,2005-01-01,13,19,50000.0,Illinois Emergency Management Agency,50000.0
2484,2011-01-01,3,23,42000.0,Illinois Community College Board,42000.0
2485,2015-01-01,17,23,26700.0,Illinois Community College Board,26700.0


In [10]:
# make the histogram 
nc = 20
na =20

hist2d, con_edges, ag_edges = np.histogram2d(df['Congress Dist'], df['AgencyID'], weights=df['Total Square Footage'],
                                             bins=[nc,na])

In [11]:
hist2d.shape, con_edges.shape, ag_edges.shape

((20, 20), (21,), (21,))

In [12]:
con_edges.max(), con_edges.min()

(18.0, 0.0)

In [13]:
ag_edges.max(),ag_edges.min()

(34.0, 0.0)

In [14]:
# specify bins
con_bins = np.linspace(0,18,nc+1)
ag_bins = np.linspace(0,34,na+1)

hist2d, con_edges, ag_edges = np.histogram2d(df['Congress Dist'], df['AgencyID'], weights=df['Total Square Footage'],
                                             bins=[con_bins,ag_bins])

In [15]:
# make bin centers from edges
con_centers = (con_edges[:-1] + con_edges[1:])/2
ag_centers = (ag_edges[:-1] + ag_edges[1:])/2

In [16]:
hist2d = hist2d.T

In [17]:
hist2d[hist2d <= 0] = np.nan
# because the difference among different buildings are not so obvious, so we would use log()
hist2d = np.log10(hist2d)

In [18]:
#1. define scales
col_sc = bqplot.ColorScale(scheme='RdPu', min=np.nanmin(hist2d), max=np.nanmax(hist2d))
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

#2. define axes
c_ax = bqplot.ColorAxis(scale=col_sc,orientation='vertical',side='right')
x_ax = bqplot.Axis(scale=x_sc,label='Agency ID')
y_ax = bqplot.Axis(scale=y_sc,label='Congress Dist',orientation='vertical',side='left')

#3. gridheatmap mark
heat_map = bqplot.GridHeatMap(color=hist2d, row=con_centers, column=ag_centers,
                              scales={'color':col_sc,'row':y_sc,'column':x_sc},
                             interactions = {'click':'select'},
                             anchor_style = {'fill':'blue'})

#4. interactivity
mySelectedLabel = ipywidgets.Label()
def get_data(change):
    if len(change['owner'].selected) == 1:
        j,i = change['owner'].selected[0]
        v = hist2d[j,i]
        mySelectedLabel.value = 'Total Square Footage in log = ' + str(v)
        
heat_map.observe(get_data, 'selected')
fig_heatmap = bqplot.Figure(marks=[heat_map],axes=[c_ax,y_ax,x_ax])

#5. layout
myDashBoard1 = ipywidgets.VBox([mySelectedLabel, fig_heatmap])
myDashBoard1

In [19]:
#1. define scales
col_sc = bqplot.ColorScale(scheme='RdPu', min=np.nanmin(hist2d), max=np.nanmax(hist2d))
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

#2. define axes
c_ax = bqplot.ColorAxis(scale=col_sc,orientation='vertical',side='right')
x_ax = bqplot.Axis(scale=x_sc,label='Agency ID')
y_ax = bqplot.Axis(scale=y_sc,label='Congress Dist',orientation='vertical',side='left')

#3. gridheatmap mark
heat_map = bqplot.GridHeatMap(color=hist2d, row=con_centers, column=ag_centers,
                              scales={'color':col_sc,'row':y_sc,'column':x_sc},
                             interactions = {'click':'select'},
                             anchor_style = {'fill':'blue'})

#4. interactivity
mySelectedLabel = ipywidgets.Label()
def get_data(change):
    print(change['owner'].selected)
        
heat_map.observe(get_data, 'selected')
fig_heatmap = bqplot.Figure(marks=[heat_map],axes=[c_ax,y_ax,x_ax])

#5. layout
myDashBoard1 = ipywidgets.VBox([mySelectedLabel, fig_heatmap])
myDashBoard1

## Right component: line plot
* x is the year
* y is total square footage acquired that year

In [20]:
df['Year Acquired'].min(), df['Year Acquired'].max()

(Timestamp('1753-01-01 00:00:00'), Timestamp('2019-01-01 00:00:00'))

In [21]:
x_sc = bqplot.DateScale(min=dt.datetime(1753,1,1),max=dt.datetime(2019,1,1))
y_sc = bqplot.LogScale()

x_ax = bqplot.Axis(scale=x_sc, label='Year')
y_ax = bqplot.Axis(scale=y_sc, label='Total Square Footage in Log', orientation='vertical')

i, j =19,0
# mask the data values of certain blocks
# grab these sets of data based on histogram edges
cons = [con_edges[i],con_edges[i+1]]
ags = [ag_edges[j],ag_edges[j+1]]
region_mask = ((df['Congress Dist']>=cons[0]) & (df['Congress Dist']<=cons[1]) &\
              (df['AgencyID']>=ags[0]) & (df['AgencyID']<=ags[1]))

In [22]:
cons, ags

([17.1, 18.0], [0.0, 1.7])

In [23]:
line = bqplot.Lines(x=df['Year Acquired'][region_mask],y=df['Yearly Total Square Footage'][region_mask], 
                    scales={'x':x_sc,'y':y_sc})

fig_line = bqplot.Figure(marks=[line],axes=[x_ax,y_ax])
fig_line

Figure(axes=[Axis(label='Year', scale=DateScale(max=datetime.datetime(2019, 1, 1, 0, 0), min=datetime.datetime…

## Link two plots
* so that you can select cells and that will update the square footage plot.

In [24]:
#create label
mySelectedLabel = ipywidgets.Label()

In [25]:
#create heatmap elements
#define scales
col_sc = bqplot.ColorScale(scheme='RdPu', min=np.nanmin(hist2d), max=np.nanmax(hist2d))
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

#define axes
c_ax = bqplot.ColorAxis(scale=col_sc,orientation='vertical',side='right')
x_ax = bqplot.Axis(scale=x_sc,label='Agency ID')
y_ax = bqplot.Axis(scale=y_sc,label='Congress Dist',orientation='vertical')

#gridheatmap mark
heat_map = bqplot.GridHeatMap(color=hist2d, row=con_centers, column=ag_centers,
                              scales={'color':col_sc,'row':y_sc,'column':x_sc},
                             interactions = {'click':'select'},
                             anchor_style = {'fill':'blue'})


In [26]:
#create line plot elements
x_scl = bqplot.DateScale(min=dt.datetime(1753,1,1),max=dt.datetime(2019,1,1))
y_scl = bqplot.LogScale()

ax_xcl = bqplot.Axis(scale=x_scl, label='Year')
ax_ycl = bqplot.Axis(scale=y_scl, label='Total Square Footage', orientation='vertical',side='left')

i, j =19,0
# mask the data values of certain blocks
# grab these sets of data based on histogram edges
cons = [con_edges[i],con_edges[i+1]]
ags = [ag_edges[j],ag_edges[j+1]]
region_mask = ((df['Congress Dist']>=cons[0]) & (df['Congress Dist']<=cons[1]) &\
              (df['AgencyID']>=ags[0]) & (df['AgencyID']<=ags[1]))

line = bqplot.Lines(x=df['Year Acquired'][region_mask],y=df['Yearly Total Square Footage'][region_mask], 
                    scales={'x':x_scl,'y':y_scl})


In [27]:
# link together 
def get_data(change):
    if len(change['owner'].selected) == 1:
        j,i = change['owner'].selected[0]
        v = hist2d[j,i]
        mySelectedLabel.value = 'Total Square Footage in log = ' + str(v)
        # update line plot too
        cons = [con_edges[i],con_edges[i+1]]
        ags = [ag_edges[j],ag_edges[j+1]]
        region_mask = ((df['Congress Dist']>=cons[0]) & (df['Congress Dist']<=cons[1]) &\
                      (df['AgencyID']>=ags[0]) & (df['AgencyID']<=ags[1]))
        line.x = df['Year Acquired'][region_mask]
        line.y = df['Yearly Total Square Footage'][region_mask]

heat_map.observe(get_data, 'selected')

In [28]:
# create figure objects
fig_heatmap = bqplot.Figure(marks=[heat_map],axes=[c_ax,y_ax,x_ax])
fig_line = bqplot.Figure(marks=[line],axes=[ax_xcl,ax_ycl])

In [29]:
# put it all together as adashboard
fig_heatmap.layout.min_width = '500px'
fig_line.layout.min_width = '500px'

plots = ipywidgets.HBox([fig_heatmap, fig_line])
myDashBoard = ipywidgets.VBox([mySelectedLabel, plots])
myDashBoard

## Things to think about
#### Can you keep the x and y ranges static on the line plot?
    I'm thinking about setting the minimum and maximum values on each axis, so that the line plot would not be subjective to the range of selected data points. But I think it's pretty clear for the plot to be dynamic changing according to the selected data.

#### Can you change the style?  
    1. Yes, the style of dashboard can be changed, like color scheme, font size and position. Jupyter interactive widgets have a "layout" attribute exposing a number of CSS properties that impact how widgets are laid out. For exaple, the margin[top/bottom/left/right]. 

    2. Also, there is a "style" attibute, which allows us to change the background_style of bqplot.figure, the legend_style, title_style and so on. For example, there are parameters in the line plot function, like line_style, color, marker_size and so on.